In [ ]:
import psycopg2
import torch
from sentence_transformers import SentenceTransformer
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_cloud_sql_pg import PostgresVectorStore
from config import DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD
from google.generativeai import configure, embed_content



In [31]:
# 1. Récupérer les données avec psycopg2 (classique)
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cursor = conn.cursor()
cursor.execute("SELECT id, content FROM ae_qa_table WHERE embedding IS NULL")
questions = cursor.fetchall()
cursor.close()
conn.close()


In [27]:
# Initialiser Hugging Face Embeddings (fallback)
hf_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")



In [34]:
import pandas as pd
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_cloud_sql_pg import PostgresEngine, PostgresVectorStore
from config import DB_NAME, DB_USER, DB_PASSWORD, PROJECT_ID, INSTANCE_NAME, REGION


engine = PostgresEngine.from_instance(
        project_id=PROJECT_ID,
        instance=INSTANCE_NAME,  # ex: "gen-ai-instance"
        region=REGION,           # ex: "europe-west1"
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    )

# 3. Initialiser le Vector Store
embedding_model = VertexAIEmbeddings(model_name="textembedding-gecko", project=PROJECT_ID)
vector_store = PostgresVectorStore.create_sync(
        engine=engine,
        table_name="ae_qa_table",
        embedding_service=hf_model,
        id_column="langchain_id",
        content_column="content",
        embedding_column="embedding"
    )


In [28]:
questions

[(4, 'What is (are) Glaucoma ?'),
 (15, 'How to prevent High Blood Pressure ?'),
 (16, 'What are the symptoms of High Blood Pressure ?'),
 (35, "What are the treatments for Paget's Disease of Bone ?"),
 (43, 'What is (are) Alcohol Use and Older Adults ?'),
 (53, 'What are the symptoms of Osteoarthritis ?'),
 (74, 'What are the symptoms of Anxiety Disorders ?'),
 (13862, 'What are the treatments for glycogen storage disease type III ?'),
 (116, 'What is (are) Knee Replacement ?'),
 (121, 'What is (are) Balance Problems ?'),
 (141,
  'what research (or clinical trials) is being done for Prostate Cancer ?'),
 (140, 'What are the treatments for Prostate Cancer ?'),
 (169, 'How to diagnose Osteoporosis ?'),
 (438, 'what research (or clinical trials) is being done for Psoriasis ?'),
 (444, 'What are the treatments for Psoriasis ?'),
 (447, 'What is (are) Psoriasis ?'),
 (222, 'What is (are) Hearing Loss ?'),
 (223, 'How to prevent Hearing Loss ?'),
 (224, 'What are the symptoms of Hearing Lo

In [ ]:
import psycopg2
import numpy as np

# Connexion PostgreSQL
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cursor = conn.cursor()

# Paramètres
BATCH_SIZE = 50  # Taille du batch pour optimiser les requêtes
USE_HF_FALLBACK = True  # Activer le fallback immédiat vers Hugging Face

# Récupérer toutes les questions sans embeddings
cursor.execute("SELECT id, question FROM ae_qa_table WHERE embedding IS NULL")
questions = cursor.fetchall()

if not questions:
    print("✅ Toutes les questions ont déjà des embeddings.")
else:
    print(f"🔄 Chargement de {len(questions)} questions à traiter...")

    for i in range(0, len(questions), BATCH_SIZE):
        batch = questions[i : i + BATCH_SIZE]
        batch_ids = [q[0] for q in batch]
        batch_questions = [q[1] for q in batch]

        try:
            # Essayer d'obtenir les embeddings via Vertex AI
            embedding_vectors = [hf_model.encode(q).tolist() for q in batch_questions]
        except Exception as e:
            if "429 Quota exceeded" in str(e) or "Retrying vertexai" in str(e):
                print(f"⚠️ Vertex AI dépasse le quota ! Passage immédiat à Hugging Face pour le batch {i // BATCH_SIZE + 1}.")
                embedding_vectors = [hf_model.encode(q).tolist() for q in batch_questions] 
            else:
                print(f"❌ Erreur inattendue avec Vertex AI : {e}")
                continue  # Passer au batch suivant

        # Stockage dans PostgreSQL en batch
        update_values = [(embedding_vectors[idx], batch_ids[idx]) for idx in range(len(batch))]
        cursor.executemany("UPDATE ae_qa_table SET embedding = %s WHERE id = %s", update_values)
        conn.commit()

        print(f"✅ Batch {i // BATCH_SIZE + 1} / {len(questions) // BATCH_SIZE + 1} traité.")

    print("🎯 Tous les embeddings ont été stockés.")

cursor.close()
conn.close()
print("🔚 Connexion PostgreSQL fermée.")


🔄 Chargement de 16412 questions à traiter...
✅ Batch 1 / 329 traité.
✅ Batch 2 / 329 traité.
✅ Batch 3 / 329 traité.
✅ Batch 4 / 329 traité.
✅ Batch 5 / 329 traité.
✅ Batch 6 / 329 traité.
✅ Batch 7 / 329 traité.
✅ Batch 8 / 329 traité.
✅ Batch 9 / 329 traité.
✅ Batch 10 / 329 traité.
✅ Batch 11 / 329 traité.
✅ Batch 12 / 329 traité.
✅ Batch 13 / 329 traité.
✅ Batch 14 / 329 traité.
✅ Batch 15 / 329 traité.
✅ Batch 16 / 329 traité.
✅ Batch 17 / 329 traité.
✅ Batch 18 / 329 traité.
✅ Batch 19 / 329 traité.
✅ Batch 20 / 329 traité.
✅ Batch 21 / 329 traité.
✅ Batch 22 / 329 traité.
✅ Batch 23 / 329 traité.
✅ Batch 24 / 329 traité.
✅ Batch 25 / 329 traité.
✅ Batch 26 / 329 traité.
✅ Batch 27 / 329 traité.
✅ Batch 28 / 329 traité.
✅ Batch 29 / 329 traité.
✅ Batch 30 / 329 traité.
✅ Batch 31 / 329 traité.
✅ Batch 32 / 329 traité.
✅ Batch 33 / 329 traité.
✅ Batch 34 / 329 traité.
✅ Batch 35 / 329 traité.
✅ Batch 36 / 329 traité.
✅ Batch 37 / 329 traité.
✅ Batch 38 / 329 traité.
✅ Batch 39 / 3

In [ ]:
import psycopg2
import numpy as np

# Connexion PostgreSQL
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cursor = conn.cursor()

# Paramètres
BATCH_SIZE = 50  # Taille du batch pour optimiser les requêtes
USE_HF_FALLBACK = True  # Activer le fallback immédiat vers Hugging Face

# Récupérer toutes les questions sans embeddings
cursor.execute("SELECT drug FROM ae_med_table WHERE embedding IS NULL")
questions = cursor.fetchall()

if not questions:
    print("✅ Toutes les drugs ont déjà des embeddings.")
else:
    print(f"🔄 Chargement de {len(questions)} drug à traiter...")

    for i in range(0, len(questions), BATCH_SIZE):
        batch = questions[i : i + BATCH_SIZE]
        batch_ids = [q[0] for q in batch]
        batch_questions = [q[1] for q in batch]

        try:
            # Essayer d'obtenir les embeddings via Vertex AI
            embedding_vectors = [hf_model.encode(q).tolist() for q in batch_questions]
        except Exception as e:
            if "429 Quota exceeded" in str(e) or "Retrying vertexai" in str(e):
                print(f"⚠️ Vertex AI dépasse le quota ! Passage immédiat à Hugging Face pour le batch {i // BATCH_SIZE + 1}.")
                embedding_vectors = [hf_model.encode(q).tolist() for q in batch_questions] 
            else:
                print(f"❌ Erreur inattendue avec Vertex AI : {e}")
                continue  # Passer au batch suivant

        # Stockage dans PostgreSQL en batch
        update_values = [(embedding_vectors[idx], batch_ids[idx]) for idx in range(len(batch))]
        cursor.executemany("UPDATE ae_med_table SET embedding = %s WHERE id = %s", update_values)
        conn.commit()

        print(f"✅ Batch {i // BATCH_SIZE + 1} / {len(questions) // BATCH_SIZE + 1} traité.")

    print("🎯 Tous les embeddings ont été stockés.")

cursor.close()
conn.close()
print("🔚 Connexion PostgreSQL fermée.")
